In [2]:
import pandas as pd
import os
from pathlib import Path
import re

In [21]:
def get_paths():
    tracts = []
    paths = []
    for path, subdirs, files in os.walk(Path(f'data/mass_tracts/')):
        tracts = subdirs
        for name in files:
            file_path = os.path.join(path,name)
            if name[0] != '.':
                paths.append(file_path)
    return paths

In [22]:
def parse(path):
    title = path.split('/')[-1][:-4].replace('_',' ')
    df = pd.read_csv(path)
    category = list(df.iloc[0])[1:]
    print(path)
    df = df.set_index('Tracts').T.filter(list(df['Tracts'][1:-2]))
    df.index = [i.split(',')[0] for i in df.index]
    df.index = [list(df.index)[list(df.index).index(i)-1] if i[:7] == 'Unnamed' else i for i in df.index]
    df.index = [list(df.index)[list(df.index).index(i)-1] if i[:7] == 'Unnamed' else i for i in df.index]
    df['Category']=category
    output = f'data/parsed_mass_tracts/{title}.csv'
    df.to_csv(output)
    return df

In [23]:
try:
    os.mkdir('data/parsed_mass_tracts/')
except:
    print('dir exists')
for path in get_paths():
    parse(path)

dir exists
data/mass_tracts/Renter_Costs_as_a_Percentage_of_Household_Income_by_Income_Level_in_the_Past_12_Months_5YR_ACS_2015-2019.csv


In [36]:
def compare_target(file, target):
    df = pd.read_csv(file)
    df.groupby('Category').sum()
    targetDF = df[df['Category']==target].dropna(axis=1, how='any').set_index('Unnamed: 0')
    totalDF = df[df['Category']=='Total'].dropna(axis=1, how='any').set_index('Unnamed: 0')
    print(totalDF)
    for t in list(targetDF.index):
        targetDF[t+' %'] = targetDF[t]/totalDF[t]
    dropcols = [c if c[-1] != '%' else None for c in list(targetDF.columns)]
    dropcols = list(filter((None).__ne__, dropcols))
    targetDF = targetDF.drop(columns=dropcols)
    try:
        os.mkdir('data/parsed_mass_tracts')
    except:
        pass
    targetDF.to_csv(f'data/parsed_mass_tracts/percent_{target}.csv')
    return targetDF, totalDF

In [38]:
targetDF, totalDF = compare_target(
    'data/parsed_mass_tracts/Renter Costs as a Percentage of Household Income by Income Level in the Past 12 Months 5YR ACS 2015-2019.csv',
    'Renter Costs >= 30% Household Income'
)

                     Less than $20,000  $20,000 to $34,999  \
Unnamed: 0                                                   
Census Tract 711.01                510                 168   
Census Tract 804.01                645                 132   
Census Tract 801                   197                 134   
Census Tract 708                   303                 134   
Census Tract 101.04                324                 100   
Census Tract 104.04                336                 165   
Census Tract 104.05                375                 124   
Census Tract 107.01                 99                  36   
Census Tract 104.03                548                 338   
Census Tract 108.02                143                  24   
Census Tract 105                   381                  83   
Census Tract 805                   531                 242   
Census Tract 709                   327                 124   

                     $35,000 to $49,999  $50,000 to $74,999  $75,000 

KeyError: 'Census Tract 711.01'